In [9]:
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import WordNetError
from nltk import word_tokenize, pos_tag   ##implementation of similarity() from http://nlpforhackers.io/wordnet-sentence-similarity/
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
def pathsim(ss1, ss2):
    if ss1.path_similarity(ss2) is None:
        return 0
    return ss1.path_similarity(ss2)
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    NoneType = type(None)
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence1nv = []
    sentence2 = pos_tag(word_tokenize(sentence2))
    sentence2nv = []
    # Get the synsets for the tagged words
    synsets1 = []
    count1 = 0
    count2=0
    tup = ()
    synsetnone1 = []
    for tagged_word in sentence1:
        if type(tagged_to_synset(*tagged_word))== NoneType and tagged_word[0][0].isupper() and penn_to_wn(tagged_word[1]) == 'n':
            count1 += 1
            tup = (tagged_word[0], tagged_word[1])
            sentence1nv.append(tup)
            synsetnone1.append(tup)
            continue
        if penn_to_wn(tagged_word[1]) == 'n':
            count1 += 1
            tup = (tagged_word[0], tagged_word[1])
            sentence1nv.append(tup)
    synsets2 = []
    tup = ()
    synsetnone2 = []
    for tagged_word in sentence2:
        if type(tagged_to_synset(*tagged_word))== NoneType and tagged_word[0][0].isupper() and penn_to_wn(tagged_word[1]) == 'n':
            count2+=1
            tup = (tagged_word[0], tagged_word[1])
            sentence2nv.append(tup)
            synsetnone2.append(tup)
            continue
        if penn_to_wn(tagged_word[1]) == 'n':
            count2+=1
            tup = (tagged_word[0], tagged_word[1])
            sentence2nv.append(tup)
    synsets1 = [tagged_to_synset(tagged_word, tag) for tagged_word, tag in sentence1nv]
    synsets2 = [tagged_to_synset(tagged_word, tag) for tagged_word, tag in sentence2nv]
    
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if not type(ss) == NoneType]
    synsets2 = [ss for ss in synsets2 if not type(ss) == NoneType]
    score = 0.0
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        if not synsets2 == []:
            best_score = max([pathsim(synset, ss) for ss in synsets2])
        # Check that the similarity could have been computed
            if best_score is not None:
                score += best_score
    for synset in synsets2:
        # Get the similarity value of the most similar word in the other sentence
        if not synsets1 == []:
            best_score = max([pathsim(synset, ss) for ss in synsets1])
        # Check that the similarity could have been computed
            if best_score is not None:
                score += best_score
    for synsetnone in synsetnone2:
        if synsetnone in synsetnone1:
            score += 1
    for synsetnone in synsetnone1:
        if synsetnone in synsetnone2:
            score += 1
    # Average the values
    score /= (count1 + count2)
    return score

In [16]:
def namedentity():
    sent = tb.tagged_sents()[22]
    print()

In [17]:
namedentity()

(S
  The/DT
  (GPE U.S./NNP)
  is/VBZ
  one/CD
  of/IN
  the/DT
  few/JJ
  industrialized/VBN
  nations/NNS
  that/WDT
  *T*-7/-NONE-
  does/VBZ
  n't/RB
  have/VB
  a/DT
  higher/JJR
  standard/NN
  of/IN
  regulation/NN
  for/IN
  the/DT
  smooth/JJ
  ,/,
  needle-like/JJ
  fibers/NNS
  such/JJ
  as/IN
  crocidolite/NN
  that/WDT
  *T*-1/-NONE-
  are/VBP
  classified/VBN
  *-5/-NONE-
  as/IN
  amphobiles/NNS
  ,/,
  according/VBG
  to/TO
  (PERSON Brooke/NNP T./NNP Mossman/NNP)
  ,/,
  a/DT
  professor/NN
  of/IN
  pathlogy/NN
  at/IN
  the/DT
  (ORGANIZATION University/NNP)
  of/IN
  (PERSON Vermont/NNP College/NNP)
  of/IN
  (GPE Medicine/NNP)
  ./.)


In [34]:
import pickle
import re
import spacy
from get_info import Course
def hasclass(question):
    courses = pickle.load(open("courses.txt", "rb"))
    nlp = spacy.load("en")

   # print(nlp(question))
    course = nlp(question).ents[0].text

    answer_course = None

    for c in courses:
        if c.title == course:
            answer_course = c
    if answer_course!=None :
        return True
    return False

In [2]:
def hasfaculty(question):
    if "Dr." in question:
        return True
    return False

In [3]:
def hasbuilding(question):
    if "hall" in question:
        return True
    return False

In [4]:
def hasevent(question):
    if "event" in question:
        return True
    return False

In [5]:
def hasdate(question):
    if "date" in question:
        return True
    return False

In [6]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import treebank as tb
from nltk.corpus.reader.wordnet import WordNetError
from nltk import word_tokenize, pos_tag 

def categorize_questions(question):
    
    categories=["Location","Instructor","Class Time","Classroom","Office Location","Office Hours","Building Hours","Contact","Others"]
    #maxsimilarity = 0
    #tags = pos_tag(word_tokenize(question))
    #print(nltk.ne_chunk(tags))
    ##building hours:
    if "When" in question or "when" in question or "time" in question: #1.1 & 1.3 & 1.4 & 1.5, 1.2
        if hasbuilding(question):
            return categories[6]
    if "open" in question or "close" in question or "closed" in question:
        if "Why" in question or "How" in question:
            return categories[8]
        return categories[6]
    if "building hours" in question:
        return categories[6]
    ##instructor
    if "instructor" in question or "teacher" in question or "teaches" in question: #2.1 2.2 2.3 class info-instructor=who+class title
   #     for word in tags:
   #         if pos_tag(word)[1]==
        if hasclass(question):
            return categories[1]
    ##class time
    if "what" in question or "What" in question:
        if "time" in question:
            if hassclass(question):
                return categories[2] ##3.1
    if "when" in question or "When" in question:
        if hasclass(question):
            return categories[2] ##3.2 3.3
    ##classroom location
    if "Where" in question or "where" in question or "location" in question or "Which building" in question:
        if hasclass(question):
            return categories[3] ##4.1 4.2 4.3
    ##faculty office location
    if "Where" in question or "where" in question or "location" in question:
        if hasfaculty(question):
            if "from" in question:
                return categories[8]
            return categories[4] ##5.1 5.2
        return categories[0]
    ##office hours
    if "office hours" in question or "When" in question and "office" in question:
        return categories[5]##6.2
    ##contact
    if "contact" in question or "email" in question or "phone number" in question:
        if hasfaculty(question):
            return categories[7] ##7.1 7.2
    if "When" in question:
        if hasevent(question):
            return "Calendar"
    if "event" in question:
        if hasdate(question):
            return "Calendar"
    
    #for sample in samples:
    #    similarity = sentence_similarity(question,sample)
    #    if similarity > maxsimilarity:
    #        maxsimilarity = similarity
    #        category = sample
    return categories[8]

In [84]:
print(categorize_questions("Who's the instructor of cse 4001"))
print(categorize_questions("who teaches cse 4001"))

Instructor
Instructor


In [85]:
print(categorize_questions("What's the location of the dining hall"))
print(categorize_questions("Where is the dining hall"))

Location
Location


In [86]:
print(categorize_questions("When is the dining hall open"))
print(categorize_questions("the building hours of the dining hall"))
print(categorize_questions("is the dining hall open now?"))
print(categorize_questions("Why is the dining hall closed now?"))

Building Hours
Building Hours
Building Hours
Others


In [87]:
print(categorize_questions("Where is Dr. John Doe"))
print(categorize_questions("location of Dr. Chan's office"))
print(categorize_questions("Where is Dr. John Doe from"))

Office Location
Office Location
Others


In [88]:
print(categorize_questions("Dr. John Doe's office hours"))
print(categorize_questions("When can I go to Dr. XX's office?"))

Office Hours
Office Hours


In [89]:
print(categorize_questions("What's Dr. Shoaff's phone number"))
print(categorize_questions("What's Dr. Shoaff's email address"))
print(categorize_questions("how can I contact Dr. John Doe"))

Contact
Contact
Contact


In [9]:
print(categorize_questions("What is Dr.Stansifer's phone number"))

Contact


In [10]:
print(categorize_questions("is wds@fit.edu Dr.Shoaff's email address?"))

Contact


In [90]:
print(categorize_questions("How can I log in to TRACKS"))

Others


In [92]:
print(categorize_questions("location of the classroom of cse 4001"))

print(categorize_questions("Where is cse 4001"))
print(categorize_questions("can I bring food into the classroom of cse 4001"))

Classroom
Classroom
Others


In [8]:
print(categorize_questions("When is the XX event"))
print(categorize_questions("what event is on date xx"))

Calendar
Calendar


In [7]:
def answer_location(question):
    return "location"

In [32]:
import pickle
import re
import spacy
from get_info import Course
def answer_class(question,subtype):
    courses = pickle.load(open("courses.txt", "rb"))
    nlp = spacy.load("en")

    #print(nlp(question))
    course = nlp(question).ents[0].text

    answer_course = None

    for c in courses:  ##index(hashtable(unique key--ID,multiple indices,can be course number,building number,teacher,+value, can ask database to do it), tree) on top of data
        if c.title == course:
            answer_course = c
    #print(answer_course)
    if subtype == "Instructor":
        return answer_course.instructor
    elif subtype == "Classroom":
        return answer_course.place
    elif subtype == "Class Time":
        return answer_course.days + " " + answer_course.time
    else:
        return answer_course.__dict__

In [10]:
def answer_staff(question,subtype):
    return "staff"

In [11]:
def answer_buildinghours(question):
    return "hours"

In [12]:
def answer_frompassage(question):
    return "passage"

In [13]:
def exists(question):
    return False;

In [14]:
def get_answer(question):
    return "old"

In [15]:
from get_info import Course
def answer(question):
    if exists(question):
        answer=get_answer(question)
        return answer
    cat = categorize_questions(question)
    if cat=="Location":
        answer = answer_location(question)
    elif cat=="Instructor" or cat=="Class Time" or cat=="Classroom":
        answer = answer_class(question,cat);
    elif cat=="Office Location" or cat=="Office Hours" or cat=="Contact":
        answer = answer_staff(question,cat);
    elif cat=="Building Hours":
        answer = answer_buildinghours(question);
    else:
        answer = answer_frompassage(question);
    return answer

In [37]:
print(answer("Who's the instructor of Intro to Human Factors"))
print(answer("Where is the classroom of Intro to Human Factors"))
print(answer("Who teaches Intro to Human Factors"))
print(answer("When is Intro to Human Factors"))
print(answer("Can you bring food into the classroom of Intro to Human Factors"))

Mehta
SKU202
Mehta
MWF 1300-1350
passage


In [38]:
print(answer("Who's the instructor of Intro to Human Factors"))

Mehta
